# Coursera Capstone Project

> The Battle of Neighborhoods in Singapore

In [1]:
import pandas as pd
import folium
import requests

print('Libraries imported.')

Libraries imported.


## Data Acquisition and Cleaning

In [2]:
# Reading the LRT Stations JSON file extracted from OneMap.sg with specific columns required
df = pd.read_csv('lrt_stations.csv')
df = df[['Station', 'Station Name', 'LATITUDE', 'LONGITUDE']]
# Extract all the Punggol LRT stations with Station Codes starting from PE or PW
df_punggol_lrt = df[df['Station'].str.contains("PE|PW")].reset_index(drop=True)
df_punggol_lrt

,Station,Station Name,LATITUDE,LONGITUDE
0,PE1,Cove,1.399282,103.905962
1,PE2,Meridian,1.396912,103.908950
2,PE3,Coral Edge,1.393909,103.912581
3,PE4,Riviera,1.394524,103.916166
4,PE5,Kadaloor,1.399509,103.916575
5,PE6,Oasis,1.402287,103.912727
6,PE7,Damai,1.405235,103.908603
7,PW1,Sam Kee,1.409613,103.904832
8,PW2,Teck Lee,1.412771,103.906578
9,PW3,Punggol Point,1.416849,103.906651


## Exploratory Data Analysis

### Plot a map of Punggol neighborhood together with LRT Stations

In [3]:
map = folium.Map(location=[df_punggol_lrt['LATITUDE'].mean(), df_punggol_lrt['LONGITUDE'].mean()], zoom_start=14)
df_punggol_lrt.apply(lambda row:folium.CircleMarker(location=[row["LATITUDE"], row["LONGITUDE"]]).add_to(map), axis=1)
map

## Connect to FourSquare API

### Setup Variables

In [1]:
CLIENT_ID = '<null>' # your Foursquare ID
CLIENT_SECRET = '<null>' # your Foursquare Secret
LIMIT = 100
VERSION = '20200531'

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: <null>
CLIENT_SECRET:<null>


### Function to get nearby venues in the neighborhood within 350 metres

In [5]:
def getNearbyVenues(station_code, station_name, lat1, long1, radius=350):
    
    venues_list = []
    
    for code, name, lat, lng in zip(station_code, station_name, lat1, long1):

        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&radius={}&limit={}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng, 
            radius, 
            LIMIT, 
            VERSION)
    
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(
            code,
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
            'Station Name', 
            'Latitude', 
            'Longitude', 
            'Venue', 
            'Venue Latitude', 
            'Venue Longitude', 
            'Venue Category']
    
    return(nearby_venues)

### Retrieve all nearby venues within each LRT station

In [6]:
result = getNearbyVenues(station_code=df_punggol_lrt.Station, 
    station_name=df_punggol_lrt["Station Name"], 
    lat1=df_punggol_lrt.LATITUDE, 
    long1=df_punggol_lrt.LONGITUDE)
result

,Station,Station Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PE1,Cove,1.399282,103.905962,Bus Stop 65079 (Blk 102C),1.398968,103.904671,Bus Station
1,PE1,Cove,1.399282,103.905962,7-ELEVEN®,1.399662,103.903849,Convenience Store
2,PE2,Meridian,1.396912,103.908950,Master Crab @ Kopitiam,1.396699,103.908607,Chinese Restaurant
3,PE3,Coral Edge,1.393909,103.912581,Ice Talk Xpress,1.394152,103.912813,Dessert Shop
4,PE3,Coral Edge,1.393909,103.912581,McDonald's,1.393536,103.913326,Fast Food Restaurant
...,...,...,...,...,...,...,...,...
76,PW5,Nibong,1.411870,103.900314,Kimly Coffeeshop,1.411619,103.897677,Coffee Shop
77,PW6,Sumang,1.408452,103.898558,Jewel Bridge,1.409577,103.895859,Bridge
78,PW6,Sumang,1.408452,103.898558,Bangkok Street Mookata,1.410788,103.900187,BBQ Joint
79,PW6,Sumang,1.408452,103.898558,Sheng Siong Supermarket,1.411038,103.900166,Market


### Total of unique categories for each station

#### From the results, it appears that Riviera is the winner, Sam Kee is the 2nd best and Oasis is the 3rd best

In [7]:
result_unique = result.groupby(['Station', 'Station Name'])[('Venue Category')].nunique().reset_index().rename(columns={'Venue Category':'Total'})
result_unique.sort_values('Total', ascending=False)

,Station,Station Name,Total
3,PE4,Riviera,16
7,PW1,Sam Kee,12
5,PE6,Oasis,9
2,PE3,Coral Edge,7
8,PW2,Teck Lee,7
4,PE5,Kadaloor,5
11,PW5,Nibong,4
10,PW4,Samudera,3
12,PW6,Sumang,3
0,PE1,Cove,2


#### Detail Venues near Riviera LRT

In [8]:
result_pe4 = result.loc[result['Station'] == 'PE4']
result_pe4.drop(['Station', 'Station Name', 'Latitude', 'Longitude'], axis=1)

,Venue,Venue Latitude,Venue Longitude,Venue Category
10,Punggol Promenade Riverside Walk,1.393244,103.917342,Trail
11,Big Fish Small Fish,1.393354,103.917749,Fish & Chips Shop
12,Punggol Golf Range,1.395727,103.917166,Golf Course
13,Whisk & Paddle,1.392919,103.917341,Café
14,Hai Bin Prawning,1.392430,103.916824,Fishing Spot
15,Container Eateries,1.393268,103.917669,Gastropub
16,Punggol Billiards,1.392766,103.916689,Pool Hall
17,Sing See Soon Floral & Landscape Pte Ltd,1.393469,103.915614,Flower Shop
18,McDonald's,1.393536,103.913326,Fast Food Restaurant
19,@PUNGGOL,1.392969,103.917187,Recreation Center


In [9]:
map = folium.Map(location=[result_pe4['Latitude'].mean(), result_pe4['Longitude'].mean()], zoom_start=17)
result_pe4.apply(lambda row:folium.CircleMarker(location=[row["Venue Latitude"], row["Venue Longitude"]]).add_to(map), axis=1)
map

> Though it has alot of entertainment venues, it appears to be lacking supermarket

#### Detail Venues near Sam Kee LRT

In [10]:
result_pw1 = result.loc[result['Station'] == 'PW1']
result_pw1.drop(['Station', 'Station Name', 'Latitude', 'Longitude'], axis=1)

,Venue,Venue Latitude,Venue Longitude,Venue Category
49,Punggol Waterway Park,1.410227,103.906975,Park
50,i Darts LAVA,1.410514,103.906188,Sports Bar
51,McDonald's,1.410088,103.905930,Fast Food Restaurant
52,The Spa @ SAFRA Punggol,1.410827,103.905034,Spa
53,MANEKINEKO,1.410295,103.905725,Music Venue
54,Heavenly Wang,1.410327,103.905959,Café
55,Sam Kee LRT Station (PW1),1.409703,103.904905,Light Rail Station
56,Jack's Place,1.410777,103.906194,Steakhouse
57,Punggol Discovery Cube,1.407816,103.903538,Plaza
58,Starker Q,1.407480,103.903197,Bistro


In [11]:
map = folium.Map(location=[result_pw1['Latitude'].mean(), result_pw1['Longitude'].mean()], zoom_start=17)
result_pw1.apply(lambda row:folium.CircleMarker(location=[row["Venue Latitude"], row["Venue Longitude"]]).add_to(map), axis=1)
map

> However, it doesn't appear to have any residental housing nearby

#### Detail Venues near Oasis LRT

In [12]:
result_pe6 = result.loc[result['Station'] == 'PE6']
result_pe6.drop(['Station', 'Station Name', 'Latitude', 'Longitude'], axis=1)

,Venue,Venue Latitude,Venue Longitude,Venue Category
36,NTUC FairPrice,1.403051,103.913420,Supermarket
37,Starbucks,1.402571,103.912896,Coffee Shop
38,McDonald’s / McCafé,1.402539,103.913043,Fast Food Restaurant
39,Wave Bridge,1.403451,103.913496,Bridge
40,KFC,1.402961,103.913430,Fast Food Restaurant
41,Popular / UrbanWrite,1.402097,103.913465,Bookstore
42,Lao Jiang 老江 Superior Soup,1.402918,103.913350,Soup Place
43,Bus Stop 65319 (Oasis Stn),1.402436,103.912363,Bus Stop
44,Rise & Grind Coffee Co,1.403183,103.913358,Coffee Shop
45,Yu Kee 友記 Specialties,1.402909,103.913325,Malay Restaurant


In [13]:
map = folium.Map(location=[result_pe6['Latitude'].mean(), result_pe6['Longitude'].mean()], zoom_start=17)
result_pe6.apply(lambda row:folium.CircleMarker(location=[row["Venue Latitude"], row["Venue Longitude"]]).add_to(map), axis=1)
map

> It has supermarket but lack of entertainment venues